# Get_Template: 
This script constructs a very simple `template` of a miyake event. The sinusoidal component of the production function is ignored resulting in a smooth curve. The template is constructed using the best fitting `model`-`dataset` pair  from the 774AD year. `emcee` is used to fit the parameters of the production function which is then used to generate the `.csv` file containing the `dc14` concentrations over just the event.

In [79]:
from os import getcwd
from ticktack.fitting import SingleFitter
from ticktack import load_presaved_model
from jax.numpy import array, pi, mean, arange, full

In [68]:
carbon_box_model = load_presaved_model(     # Generating the CarbobBoxModel using ticktack
    "Guttler14",                            # Selecting an arbitrary model 
    production_rate_units="atoms/cm^2/s"    # Units of the model
)

bayesian_model = SingleFitter(carbon_box_model)   # Fitting a model 
bayesian_model.prepare_function(model="simple_sinusoid") # Generating the simple sin model
bayesian_model.load_data(f"{getcwd()}/datasets/775AD/NH/Miyake12_Cedar.csv")   

mcmc_samples = bayesian_model.MarkovChainSampler(
    array([775., 1./12, pi/2., 81./12]),    # Initial guess of the parameters
    likelihood=bayesian_model.log_joint_simple_sinusoid,    
    args=(array([770., 0., -pi, 0.]), array([780., 5., pi, 15.])) # Upper and lower bounds
)   

Running burn-in...


100%|██████████| 500/500 [00:41<00:00, 11.94it/s]


Running production...


100%|██████████| 1000/1000 [01:27<00:00, 11.38it/s]


In [76]:
# concentrations, final_concentrations = carbon_box_model.run(
#     time_out=arange(750, 800), 
#     oversample=108, 
#     production=bayesian_model.production, 
#     steady_state_production=1.88,
#     args = mean(mcmc_samples, axis=0)
# )

In [80]:
bayesian_model.annual = arange(750, 800)
bayesian_model.mask = full((50,), 1)
bayesian_model.dc14(mean(mcmc_samples, axis=0))